In [77]:
from aocd import get_data
data = get_data(day=16, year=2023)

In [34]:
import numpy as np
from dataclasses import dataclass, replace, field

In [2]:
with open('.day16.txt', 'r') as f:
    data = f.read().strip()

In [78]:
board = np.array([ list(x) for x in data.split('\n') ])
energy = np.zeros(board.shape, dtype=int)
for r in board:
    print(''.join(r))

\.........-......|...../................................-...........|.../.\\...\|...................|.........
../..\...............-../............\.........-......-................././........|......................|...
......../..-........../................../...|.....-......................\.....-..............|............/|
......-......-............................\..\............../................./.-.................\.....|.\|..
...-....-|....................-...-..../...................-....../...........|.............\.....-...........
..\............../..../................|...........|/......|..........-\.........|...../....................|.
\...........\.\....../...|..........--.................|......//...../..-.....|.....-.........................
..................................................|...........\.\.................../.....................\...
.....-...................-............./.......................|............/...|..........................-..
.

In [124]:
@dataclass(frozen=True)
class Vector:
    y: int = field(compare=True)
    x: int = field(compare=True)
    d: str = field(compare=True)
    start: Vector = field(default=None, compare=False)
    
def is_in(b, y, x):
    return  y >= 0 and x >= 0 and y < (b.shape[0]) and x < (b.shape[1])
    
def next(v):
    match v.d:
        case 'N': return (v.y-1, v.x)
        case 'E': return (v.y, v.x+1)
        case 'S': return (v.y+1, v.x)
        case 'W': return (v.y, v.x-1)

def vec(y, x, d, **kwargs):
    sv = Vector(y, x, d)
    if 'start' not in kwargs:
        kwargs['start'] = replace(sv, start=None)
    return replace(sv, **kwargs)

def shoot(b, e, v:Vector, q:list, cache:dict):
    if not is_in(b, v.y, v.x):
        #print('addin outta bound cache', v.start, '=', replace(v, start=None))
        cache[replace(v.start)] = replace(v)
        return
    elif v.start in cache:
        return
    e[v.y,v.x] = 1
    c = b[v.y,v.x]
    if c == '\\':
        if v.d == 'E':
            v = replace(v, d='S')
        elif v.d == 'W':
            v = replace(v, d='N')
        elif v.d == 'N':
            v = replace(v, d='W')
        elif v.d == 'S':
            v = replace(v, d='E')
    elif c == '/':
        if v.d == 'E':
            v = replace(v, d='N')
        elif v.d == 'W':
            v = replace(v, d='S')
        elif v.d == 'N':
            v = replace(v, d='E')
        elif v.d == 'S':
            v = replace(v, d='W')
    elif c == '|':
        if v.d in ['E','W']:
            #print('addin EW cache', v.start, '=', replace(v, start=None))
            cache[replace(v.start)] = replace(v)
            q.append(vec(v.y, v.x, 'N'))
            q.append(vec(v.y, v.x, 'S'))
            return
    elif c == '-':
        if v.d in ['N','S']:
            #print('addin NS cache', v.start, '=', replace(v, start=None))
            cache[replace(v.start)] = replace(v)
            q.append(vec(v.y, v.x, 'E'))
            q.append(vec(v.y, v.x, 'W'))
            return
    ny, nx = next(v)
    shoot(b, e, replace(v, y=ny, x=nx), q, cache)

def nrg(board, y, x, d, cache):
    cache = {}
    energy = np.zeros(board.shape, dtype=int)
    queue = [vec(y, x, d)]
    while queue:
        shoot(board, energy, queue.pop(), queue, cache)

    #print(np.where(energy == 1))
    return energy.sum()

cache = {}
nrg(board, 0, 0, 'E', cache)
#print('queue', queue)
#print('cache', cache)

7860

In [128]:
max_nrg = 0
for y in range(board.shape[0]):
    #print(y, 0, 'E')
    #print(y, board.shape[1]-1, 'W')
    s = max(nrg(board, y, 0, 'E', cache), nrg(board, y, board.shape[1]-1, 'W', cache))
    if s > max_nrg:
        max_nrg = s
for x in range(board.shape[1]):
    #print(0, x, 'S')
    #print(board.shape[0]-1, x, 'N')
    s = max(nrg(board, 0, x, 'S', cache), nrg(board, 0, x, 'S', cache))
    if s > max_nrg:
        max_nrg = s
max_nrg

8331